In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import LSTM, Dense

# Raw data loading

In [ ]:
raw_df = pd.read_csv('car_data.csv')
print(raw_df.shape)

In [ ]:
raw_df.head()

# Data transform

In [ ]:
df = raw_df.drop('timestamp', axis=1)

raw_X = df[['use_percent','number_of_process','is_recording','framerate','resolution_height','resolution_width','speed','red_intensity','green_intensity','blue_intensity','is_buzzing','ultrasonicOn']]
raw_Y = df['voltage']

scaler = MinMaxScaler()
raw_X = scaler.fit_transform(raw_X)

all_X = np.array(raw_X)
all_Y = np.array(raw_Y)

X_train, X_test, Y_train, Y_test = train_test_split(all_X, all_Y, test_size=0.25)

X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

# Creating model

In [ ]:
def SoC_forecasting_model(input_shape, loss='mse', optimizer='rmsprop', verbose=True):
    SoC_model = Sequential(name ='SoC_Forecasting_Model')
    SoC_model.add(LSTM(input_shape[1], input_shape=(input_shape[1], input_shape[2]), recurrent_dropout=0.2, return_sequences=True, name='LSTM_input'))
    SoC_model.add(LSTM(32, recurrent_dropout=0.2, name='LSTM_hidden'))
    SoC_model.add(Dense(input_shape[2], name='Dense_output'))
    SoC_model.compile(loss=loss, optimizer=optimizer)
    if verbose :
        print(SoC_model.summary())
    return SoC_model

In [ ]:
model = SoC_forecasting_model(X_train.shape)

In [ ]:
history = model.fit(X_train,Y_train, epochs=10, batch_size=16, validation_data=(X_test, Y_test), shuffle=False)

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
mse = model.evaluate(X_test, Y_test)
print('MSE : ', mse)

In [ ]:
model.save('4WD_Car_SoC_Forecasting_model_deep')